# Analyzing Stock Sentiment from Twits


In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F



## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data


In [3]:
"""print out the number of twits"""

print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data


### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
  
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = text.replace('$',' ')
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@([A-Za-z0-9_]+)', ' ', text)
    
    # Replace everything not a letter with a space
    text = re.sub('[^a-zA-Z]',' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = nltk.tokenize.word_tokenize(text)
    
    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(t) for t in tokens]
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
import nltk
nltk.download('punkt')

tokenized = [preprocess(m) for m in (messages)] 

print(tokenized)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

wl = []
for t in tokenized:
    for w in t:
        wl.append(w)

bow = Counter(wl)
#print(bow)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""


# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {k:v/len(tokenized) for k, v in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff =  0.000006

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 9

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = dict(bow.most_common(high_cutoff))

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]

len(filtered_words) 

21649

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab =  {k: v for v, k in enumerate(filtered_words,1)}

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = dict([(v,k) for k, v in vocab.items()])

# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in words if word in vocab] for words in tokenized]

print(filtered[:15])

[['fitb', 'great', 'buy', 'at', 'ill', 'wait'], ['msft'], ['staanalystalert', 'tdg', 'jefferies', 'maintains', 'with', 'rating', 'hold', 'setting', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'buy'], ['i', 'heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'stocktwits'], ['reveal', 'yourself'], ['aapl', 'why', 'drop', 'i', 'warren', 'buffet', 'taking', 'out', 'his', 'position'], ['ba', 'bear', 'have', 'reason', 'pay', 'more', 'attention', 'ba'], ['bac', 'ok', 'good', 'we', 're', 'not', 'dropping', 'in', 'price', 'over', 'weekend', 'lol'], ['amat', 'daily', 'chart', 'we', 'need', 'get', 'back', 'above'], ['gme', 'drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'if', 'bo', 'then', 'bingo', 'what', 'odds'], ['sbux', 'strong', 'buy'], ['snp', 'short', 'ratio', 'at', 'and', 'short', 'float', 'via'], ['nflx', 'price', 'squeezing', 'perfect', 'place', 'an', 'option', 'straddle', 'near', 'supportin

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [10]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [11]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19919264212902135

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [13]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout_prob = dropout
        
     

        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers,
                           dropout=self.dropout_prob, batch_first=False)
        # Setup additional layers
        self.dropout = nn.Dropout(self.dropout_prob)

        self.fc = nn.Linear(self.lstm_size, self.output_size)
        self.log_softmax = nn.LogSoftmax(dim=-1)

       
        
    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
                
        gpu_enable = True
        
        weight = next(self.parameters()).data
        if (gpu_enable):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                     weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                     weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)
        lstm_out = lstm_out[-1, :,:]

        out = self.dropout(lstm_out)
        out = self.fc(out)

        soft_out = self.log_softmax(out)
       
        
        return soft_out, hidden

### View Model

In [14]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-2.0168, -1.5067, -1.6858, -1.1745, -1.8904],
        [-2.0299, -1.5250, -1.6358, -1.2032, -1.8580],
        [-2.0200, -1.5247, -1.6831, -1.1693, -1.8755],
        [-2.0305, -1.5165, -1.7078, -1.1572, -1.8733]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [15]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation


In [16]:
"""
Split data into training and validation datasets. 
"""   


split_frac = 0.8
tot= len(token_ids)
split_idx = int(tot*split_frac)
 
train_features = token_ids[:split_idx]
train_labels = sentiments[:split_idx]
valid_features = token_ids[split_idx:]
valid_labels = sentiments[split_idx:]

In [17]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
train the neural network

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(21650, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [25]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 2
batch_size = 512
learning_rate = 0.001


print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        hidden = tuple([each.data for each in hidden])
        
        # zero accumulated gradients
        model.zero_grad()
        
        # get the output from the model
        output, hidden = model(text_batch, hidden)
        
        
        
        # calculate the loss and perform backprop
        loss = criterion(output, labels.long())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        
        if steps % print_every == 0:
            
            val_losses = []
            
            model.eval()
            #import pdb; pdb.set_trace()
            # TODO Implement: Print metrics
            for val_batch,val_labels in dataloader(
                valid_features, valid_labels, batch_size=batch_size,sequence_length=20, shuffle=True):
                # Get validation loss
                val_h = model.init_hidden(val_labels.shape[0])
                #val_h = model.init_hidden(batch_size)
            
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])
                
                val_batch,val_labels = val_batch.to(device), val_labels.to(device)
                
                for each in val_h:
                    each.to(device)
                    
                val_out, val_h = model.forward(val_batch,val_h)
                
                val_loss = criterion(val_out, val_labels.long())
                
                val_losses.append(val_loss.item())
                
                # calculate metrics:
                ps = torch.exp(val_out)
                top_p,top_class = ps.topk(1,dim=1)
                
            #print(val_losses)
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train Loss:  {loss.item()}.. "
                  f"Test loss: {val_loss.item():.3f}.. "
                 f"Test Accuracy: {ps.mean()}.."   )
            
            model.train()
        
           

Starting epoch 1
Epoch 1/2.. Train Loss:  0.6574186682701111.. Test loss: 0.695.. Test Accuracy: 0.19999998807907104..
Epoch 1/2.. Train Loss:  0.6310083270072937.. Test loss: 0.767.. Test Accuracy: 0.19999998807907104..
Epoch 1/2.. Train Loss:  0.7242069244384766.. Test loss: 0.754.. Test Accuracy: 0.19999997317790985..
Epoch 1/2.. Train Loss:  0.677431583404541.. Test loss: 0.732.. Test Accuracy: 0.20000000298023224..
Epoch 1/2.. Train Loss:  0.6868929862976074.. Test loss: 0.707.. Test Accuracy: 0.20000000298023224..
Epoch 1/2.. Train Loss:  0.6691334843635559.. Test loss: 0.775.. Test Accuracy: 0.19999997317790985..
Epoch 1/2.. Train Loss:  0.5955350399017334.. Test loss: 0.701.. Test Accuracy: 0.20000000298023224..
Epoch 1/2.. Train Loss:  0.5993708968162537.. Test loss: 0.831.. Test Accuracy: 0.20000000298023224..
Epoch 1/2.. Train Loss:  0.6902860403060913.. Test loss: 0.788.. Test Accuracy: 0.20000000298023224..
Epoch 1/2.. Train Loss:  0.7320969104766846.. Test loss: 0.728.. T

## Making Predictions
### Prediction 


In [28]:
import numpy as np

def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
   
    tokens = preprocess(text)
    # Filter non-vocab words
    tokens = [w for w in tokens if w in vocab] 
    # Convert words to ids
    tokens = [vocab[x] for x in tokens] 
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1, 1))) 
    # Get the NN output
    hidden = model.init_hidden(1) 
    logps, _ = model.forward(text_input, hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.round(logps.squeeze())
    
    prediction = torch.exp(logps)
    print(prediction)
    
    return prediction


In [29]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[ 0.0004,  0.0109,  0.0094,  0.7008,  0.2785]])


tensor([[ 0.0004,  0.0109,  0.0094,  0.7008,  0.2785]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?


#### The model prediction is "positive" and the uncertainty is 70,71% while the certainty is 29,29%


Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [30]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [31]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [32]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [33]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

tensor([[ 2.0255e-05,  1.2792e-04,  9.9983e-01,  1.3874e-05,  9.3687e-06]])
tensor([[ 0.1281,  0.7090,  0.0464,  0.1154,  0.0012]])
tensor([[ 0.0314,  0.0549,  0.6031,  0.2392,  0.0714]])


{'symbol': '$AAPL',
 'score': tensor([[ 0.0314,  0.0549,  0.6031,  0.2392,  0.0714]]),
 'timestamp': '2018-11-01T00:00:18Z'}